# Symmetrical Amplifier

In this notebook the circuit shown in the following schematic will be sized to acheive a certain gain.
The resulting procedure takes a target gain as input and can be run for any technology, for which trained models are available.

![Symmetrical Operatrional Amplifier](./fig/sym.png)



In [1]:
%matplotlib inline

In [31]:
import os
import torch
import numpy as np
import torch as pt
import pandas as pd
import joblib as jl
from functools import partial
from scipy.interpolate import pchip_interpolate, interp1d
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from precept import PreceptModule

## Specification

The overall performance of the circuit is approximated during sizing by

$$A_{0} \approx M \cdot \frac{g_{\mathrm{m}, \mathtt{d}}}{g_{\mathrm{ds},\mathtt{cm22}} + g_{\mathrm{ds},\mathtt{cm32}}}$$

and 

$$ f_{0} \approx \frac{g_{\mathrm{ds},\mathtt{cm22}} + g_{\mathrm{ds},\mathtt{cm32}}}{2 \cdot \pi \cdot C_{L}} $$

At the end these performances are verified by simulation.

| Parameter             | Specification |
|-----------------------|--------------:|
| $V_{\mathrm{DD}}$     |      $1.2\,V$ |
| $V_{\mathrm{in,cm}}$  |      $0.6\,V$ |
| $V_{\mathrm{out,cm}}$ |      $0.6\,V$ |
| $I_{\mathtt{B0}}$     |   $10\,\mu A$ |
| $C_{\mathrm{L}}$      |     $10\,p F$ |

In [52]:
V_DD  = 1.2
V_SS  = 0.0
V_ICM = 0.6
V_OCM = 0.6
I_B0  = 10e-6
C_L   = 10e-12

## Simulator Setup

[PySpice]() is used for verifying the design by simulation within this notebook.

In [53]:
import logging
from PySpice.Spice.Netlist import Circuit, SubCircuitFactory
from PySpice.Spice.Library import SpiceLibrary
from PySpice.Unit import *

The Symmetrical amplifier is setup as a subcircuit to be included into a testbench.

In [54]:
class SymAmp(SubCircuitFactory):
    NAME = "symamp"
    NODES = ("REF", "INP", "INN", "OUT", "SS", "DD")
    
    def __init__(self):
        super().__init__()
        # Biasing Current Mirror
        self.MOSFET("NCM11", "REF", "REF", "SS", "SS", model = "nmos")
        self.MOSFET("NCM12", "X"  , "REF", "SS", "SS", model = "nmos")
        # Differential Pair
        self.MOSFET("ND11" , "U"  , "INP", "X" , "SS", model = "nmos")
        self.MOSFET("ND12" , "W"  , "INN", "X" , "SS", model = "nmos")
        # PMOS Current Mirrors
        self.MOSFET("PCM221", "D"  , "U" , "DD", "DD", model = "pmos")
        self.MOSFET("PCM222", "U"  , "U" , "DD", "DD", model = "pmos")
        self.MOSFET("PCM211", "W"  , "W" , "DD", "DD", model = "pmos")
        self.MOSFET("PCM212", "OUT", "W" , "DD", "DD", model = "pmos")
        # NMOS Current Mirror
        self.MOSFET("NCM31", "D"  , "D"  , "SS", "SS", model = "nmos")
        self.MOSFET("NCM32", "OUT", "D"  , "SS", "SS", model = "nmos")

In [55]:
spice_library = SpiceLibrary("../lib/90nm_bulk.lib")
netlist = Circuit("symamp_tb")
netlist.include(spice_library["nmos"])
netlist.subcircuit(SymAmp())

In [56]:
netlist.X("sym", "symamp", "R", "P", "N", "O", 0, "DD")
symamp = list(netlist.subcircuits)[0]
i_ref  = netlist.I("ref", 0, "R", I_B0@u_uA)
v_dd   = netlist.VoltageSource("dd", "DD", 0, V_DD@u_V)
v_ip   = netlist.VoltageSource("ip", "P", 0, V_ICM@u_V)
v_in   = netlist.SinusoidalVoltageSource( "in", "N", "E"
                                        , dc_offset=0.0@u_V
                                        , ac_magnitude=-1.0@u_V
                                        , )
e_buf  = netlist.VoltageControlledVoltageSource("in", "E", 0, "O", 0, 1.0@u_V)
c_l    = netlist.C("L", "O", 0, C_L@u_pF)

The simulation function takes a `dict` of the structure:

```python
{ "<device_name>" : { "w" : <width:float>
                    , "l" : <length:float>
                    , }
, ...
, }
```

where `device_name` corresponds to each transistor in the symmetrical amplifier.

In [30]:
def simulate(device_paramters):
    for device, parameters in device_paramters.items():
        symamp.element(device).width  = parameters["w"]
        symamp.element(device).length = parameters["l"]
        
    simulator = netlist.simulator( simulator="ngspice-subprocess"
                                 , temperature=27
                                 , nominal_temperature=27
                                 , )

    logging.disable(logging.FATAL)
    analysis  = simulator.ac( start_frequency  = 1.0@u_Hz
                            , stop_frequence   = 1e11@u_Hz
                            , number_of_points = 10
                            , variation        = "dec"
                            , )
    logging.disable(logging.NOTSET)
    
    freq  = np.array(analysis.frequency)
    gain  = (20 * np.log10(np.absolute(analysis["O"]))) - (20 * np.log10(np.absolute(analysis["N"])))
    phase = np.angle(analysis["O"], deg=True) - np.angle(analysis["N"], deg=True)
    
    gf = [gain[np.argsort(gain)], freq[np.argsort(gain)]]
    pf = [phase[np.argsort(phase)], freq[np.argsort(phase)]]
    
    A0dB = pchip_interpolate(freq, gain [1.0])
    A3dB = A0dB - 3.0
    f3dB = pchip_interpolate(*gf, [A3dB])
    
    fug = pchip_interpolate(*gf, [0.0]) if A0dB > 0 else np.ones(1)
    fp0 = pchip_interpolate(*pf, [0.0])
    
    PM = pchip_interpolate(freq, phase [fug]) if A0dB > 0 else np.zeros(1)
    GM = pchip_interpolate(freq, gain, [fp0])
    
    return [p.item() for p in [A0dB, f3dB, fug, PM, GM]]

## Device Model Setup

The `PrimitiveDevice` class acts as interface to the machine learning models. With this, multiple models of different types and technologies can be instantiatede and compared.

In [109]:
class PrimitiveDevice():
    def __init__(self, prefix, params_x, params_y):
        self.prefix   = prefix
        self.params_x = params_x
        self.params_y = params_y
        
        self.model = PreceptModule.load_from_checkpoint(f"{self.prefix}.ckpt")
        self.model.cpu()
        self.model.eval()
        
        self.scale_x = jl.load(f"{self.prefix}.X")
        self.scale_y = jl.load(f"{self.prefix}.Y")
        
    # predict :: DataFame -> DataFrame
    def predict(self, X):
        with pt.no_grad():
            X_ = self.scale_x.transform(np.vstack([ X.gmid.values
                                                  , np.log10(X.fug.values)]).T)
            Y_ = self.model(pt.from_numpy(np.float32(X_))).numpy()
            Y  = pd.DataFrame( self.scale_y.inverse_transform(Y_)
                             , columns=params_y )
            Y.jd   = np.power(10, Y.jd.values)
            Y.gdsw = np.power(10, Y.gdsw.values)
        return pd.DataFrame(Y, columns=self.params_y)

In [135]:
devices           = [ "MNCM11", "MNCM12", "MND11", "MND12", "MNCM31", "MNCM32"
                    , "MPCM221" , "MPCM222", "MPCM211", "MPCM212" ]
reference_devices = [ "MNCM12", "MND12", "MPCM212", "MNCM32" ]

In [136]:
params_x = ["gmid", "fug"]
params_y = ["jd", "L", "gdsw"]

Initially the symmetrical amplifier is sized with the models for the $90\,\mathrm{nm}$ technology. 
Later this can be changed to any other technology model, yielding similar results.

In [137]:
nmos90 = PrimitiveDevice("../models/example/90nm-nmos/g-nmos-90nm", params_x, params_y)
pmos90 = PrimitiveDevice("../models/example/90nm-pmos/g-pmos-90nm", params_x, params_y)

## Design Procedure

First, the specification, given in the table above is considered.
from which a biasing current ${I_{\mathtt{B1}} = \frac{I_{\mathrm{B0}}}{2}}$ is defined.
Additionally, the ratio ${M = 1 : 4}$ of the PMOS current mirrors `MPCM2` is specified. 
Usually, this is chosen to balance power consumption and phase margin. Since this
has to be analyzed separately by simulation, starting values ${M_{\mathtt{cm21}} = 1}$ 
and ${M_{\mathtt{cm22}} = 4}$ are selected.

In [139]:
M    = 4
R    = 2
I_B1 = I_B0 / R
I_B2 = (I_B1 / 2) * M

First, since the common mode output voltage $V_{\mathrm{out,cm}} = 0.6\,\mathrm{V}$ is known, 
the NMOS current mirror `MNCM3` is considered with first.

In [132]:
def size_symamp(e_char):
    dev_char = { dev: nmos90.predict(pd.DataFrame([char])).to_dict("records")[0]
                 for dev,char in e_char.items() }
    
    dev_char["MNCM12"]["W"]  = (I_B1 / dev_char["MNCM12"]["jd"]) / R
    dev_char["MND12"]["W"]   = (I_B1 / 2) / dev_char["MND12"]["jd"]
    dev_char["MPCM212"]["W"] = I_B2 / dev_char["MPCM212"]["jd"]
    dev_char["MNCM32"]["W"]  = I_B2 / dev_char["MNCM32"]["jd"]
             
    dev_char["MNCM11"]       = dev_char["MNCM12"]
    dev_char["MNCM11"]["W"]  = dev_char["MNCM12"]["W"] * R
    
    dev_char["MND11"]   = dev_char["MND12"]
    
    dev_char["MNCM31"]  = dev_char["MNCM32"]
    
    dev_char["MPCM222"] = dev_char["MPCM212"]
    
    dev_char["MPCM211"] = dev_char["MPCM212"]
    dev_char["MPCM211"]["W"] = dev_char["MPCM212"]["W"] / M
    dev_char["MPCM221"] = dev_char["MPCM212"]
    dev_char["MPCM221"]["W"] = dev_char["MPCM212"]["W"] / M
    return dev_char

In [133]:
electrical_characteristics = { rd : { "gmid" : 10.0
                                    ,  "fug" : 1e9
                                    , }
                               for rd in reference_devices }

In [134]:
size_symamp(electrical_characteristics)

{'MNCM12': {'jd': 8.256196022033691,
  'L': 1.1270124105067225e-06,
  'gdsw': 0.2813168168067932},
 'MND12': {'jd': 8.256196022033691,
  'L': 1.1270124105067225e-06,
  'gdsw': 0.2813168168067932},
 'MPCM212': {'jd': 8.256196022033691,
  'L': 1.1270124105067225e-06,
  'gdsw': 0.2813168168067932},
 'MNCM32': {'jd': 8.256196022033691,
  'L': 1.1270124105067225e-06,
  'gdsw': 0.2813168168067932}}

In [100]:
#nmos90.predict(pd.DataFrame([electrical_characteristics["MND12"]]))
pd.DataFrame([electrical_characteristics["MND12"]])

,gmid,fug
0,10.0,1.000000e+09
